READS IN COMPILATION DATA FROM PHASE 2 AND PULLS OFF<br>
THE 10 YEAR EXPOSURES BY TIER FOR NY

# Import libraries

In [12]:
import os,sys,logging
from getpass import getpass

import sfcommon.sfdb as sf
from spcommon.amt_eval import AmtEval

import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.float_format = '{:,.4f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_column', 100)

In [13]:
#Useful Code
import base64
from IPython.display import HTML

def create_download_link( df,  filename):
    """
    This function creates link(s) for downloading pandas DF as csv file
    df = is a list of dfs that need's to be downloaded, might be list with only 1 element/ example[df1, df2]
    filename = list of strings in the same order as df list, needs to be created with .csv extension/ example['file1.csv', 'file_2.csv']
    """
    html=""
    for i in range(len(df)):
        csv = df[i].to_csv()
        b64 = base64.b64encode(csv.encode())
        payload = b64.decode()
        title = "Download CSV file '{}'".format(filename[i])
        html+= '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a> <br>'.format(payload=payload,filename=filename[i],title=title)
    
    return HTML(html)

## Useeful functions

In [14]:
import base64
from IPython.display import HTML

def create_download_link( df,  source, filename):
    """
    This function creates link(s) for downloading pandas DF as csv file
    df = is a list of dfs that need's to be downloaded, might be list with only 1 element/ example[df1, df2]
        in case of SF table the names of tables should in string type/ exmaple ['table1','table2']
    source = list value between : "PD"/"SF", pandas or SnowFlace respectively depending on the source of table
    filename = list of strings in the same order as df list, needs to be created with .csv extension/ example['file1.csv', 'file_2.csv']
    """
    print_out = '\nThe following table(s) were created and ready to be downloaded: \n'
    html=""
    if source=='PD':
        for i in range(len(df)):
            csv = df[i].to_csv(index=False)
            b64 = base64.b64encode(csv.encode())
            payload = b64.decode()
            title = "Download CSV file '{}'".format(filename[i])
            html+= '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a> <br>'.format(payload=payload,filename=filename[i],title=title)
            print_out+=filename[i]
            print_out+= 'with {rows} rows, {cols} columns\n'.format(rows=str(df[i].shape[0]),cols=str(df[i].shape[1]))
    if source=='SF':
        for i in range(len(df)):
            qry = "select * from {} ".format(df[i])
            DF= %read_sql {qry}
            csv = DF.to_csv(index=False)
            b64 = base64.b64encode(csv.encode())
            payload = b64.decode()
            title = "Download CSV file '{}'".format(filename[i])
            html+= '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a> <br>'.format(payload=payload,filename=filename[i],title=title)
            print_out+=filename[i]
            print_out+= ' with {rows} rows, {cols} columns\n'.format(rows=str(DF.shape[0]),cols=str(DF.shape[1]))
    else:
        print("SOURCE ERROR, 'source' can be one of: 'PD' or 'SF'")
    print(print_out)
    return HTML(html)

# Connection to SF

In [15]:
if os.getenv('SFPASSWORD') is "" or os.getenv('SFPASSWORD') is None:
    os.environ['SFPASSWORD'] = getpass(prompt='Password: ', stream=None)
    
conn = sf.get_connect()

<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-15-1bbc87a2bb48>:1: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if os.getenv('SFPASSWORD') is "" or os.getenv('SFPASSWORD') is None:


In [16]:
%reload_ext sql_magic
%config SQL.conn_name = 'conn'

# Parameters

In [17]:
input_table = 'IPO_DEAL_DB.SB_MORGAN_PERS_LIAB.PL_PH2MRGA1_2019_MF_OUTPUT'

latest_year = 2019
earliest_year = latest_year-9
state=31

In [18]:
from datetime import date as dt
user_id=os.getenv("SFUSER")
today = dt.today()
date=str(today)[:4]+'_'+str(today)[5:7]+'_'+str(today)[8:]

# Calculation

In [19]:
%read_sql create or replace temp table inphase2 as select * from {input_table}

Query started at 08:29:42 AM EDT; Query executed in 0.02 m

,status
0,Table INPHASE2 successfully created.


In [20]:
%%read_sql 
select distinct st, TERR_GRP, class_cd,
sum(ST_EARN_HOUSE_YR) over(partition by st, TERR_GRP, class_cd)  as ST_EARN_HOUSE_YR,
sum(CW_EARN_HOUSE_YR) over(partition by st, TERR_GRP, class_cd)  as CW_EARN_HOUSE_YR
from
    (select  REP_YR,
    CLASS_CD,
    st,
    TERR_GRP,
    (case when ST_EARN_HOUSE_YR is null then 0 else round(ST_EARN_HOUSE_YR) end) as ST_EARN_HOUSE_YR,
    (case when CW_EARN_HOUSE_YR is null then 0 else round(CW_EARN_HOUSE_YR) end) as CW_EARN_HOUSE_YR
    from inphase2 
    where rep_yr<={latest_year} and rep_yr>={earliest_year} and  st={state}
    order by st, rep_yr,TERR_GRP, class_cd)
order by 1,2,3

Query started at 08:29:43 AM EDT; Query executed in 0.01 m

,ST,TERR_GRP,CLASS_CD,ST_EARN_HOUSE_YR,CW_EARN_HOUSE_YR
0,31,00,201,0,0
1,31,01,201,11737,5866636
2,31,01,202,134,255197
3,31,01,203,498,164416
4,31,01,204,232,125716
5,31,01,211,0,49991
6,31,01,212,0,3210
7,31,01,213,0,5234
8,31,01,214,0,2241
9,31,01,221,65,280404


In [21]:
%%read_sql 
select distinct  st, CLASS_CD,
    (case when ST_EARN_HOUSE_YR is null then 0 else sum(round(ST_EARN_HOUSE_YR)) over(partition by st, CLASS_CD) end)  as ST_EARN_HOUSE_YR,
    (case when CW_EARN_HOUSE_YR is null then 0 else sum(round(CW_EARN_HOUSE_YR)) over(partition by st, CLASS_CD) end)  as CW_EARN_HOUSE_YR
from inphase2 
    where rep_yr<={latest_year} and rep_yr>={earliest_year} and st={state} and terr_grp <>0
    order by st, class_cd

Query started at 08:29:44 AM EDT; Query executed in 0.01 m

,ST,CLASS_CD,ST_EARN_HOUSE_YR,CW_EARN_HOUSE_YR
0,31,201,162348,23466544
1,31,202,119574,1020788
2,31,203,65078,657664
3,31,204,18474,502864
4,31,211,5,199964
5,31,212,0,12840
6,31,213,0,20936
7,31,214,0,8964
8,31,221,2766,1121616
9,31,222,1065,45084


# Final Outputs

## Tables creation

In [23]:
%%read_sql 
create or replace temp table totsw as 
select distinct st, TERR_GRP, class_cd,
sum(ST_EARN_HOUSE_YR) over(partition by st, TERR_GRP, class_cd)  as ST_EARN_HOUSE_YR,
sum(CW_EARN_HOUSE_YR) over(partition by st, TERR_GRP, class_cd)  as CW_EARN_HOUSE_YR
from
    (select  REP_YR,
    CLASS_CD,
    st,
    TERR_GRP,
    (case when ST_EARN_HOUSE_YR is null then 0 else round(ST_EARN_HOUSE_YR) end) as ST_EARN_HOUSE_YR,
    (case when CW_EARN_HOUSE_YR is null then 0 else round(CW_EARN_HOUSE_YR) end) as CW_EARN_HOUSE_YR
    from inphase2 
    where rep_yr<={latest_year} and rep_yr>={earliest_year} and  st={state}
    order by st, rep_yr,TERR_GRP, class_cd)
order by 1,2,3

Query started at 08:30:55 AM EDT; Query executed in 0.13 m

,status
0,Table TOTSW successfully created.


In [24]:
%%read_sql
create or replace temp table tot as 
select distinct  st, CLASS_CD,
    (case when ST_EARN_HOUSE_YR is null then 0 else sum(round(ST_EARN_HOUSE_YR)) over(partition by st, CLASS_CD) end)  as ST_EARN_HOUSE_YR,
    (case when CW_EARN_HOUSE_YR is null then 0 else sum(round(CW_EARN_HOUSE_YR)) over(partition by st, CLASS_CD) end)  as CW_EARN_HOUSE_YR
from inphase2 
    where rep_yr<={latest_year} and rep_yr>={earliest_year} and st={state} and terr_grp <>0
    order by st, class_cd

Query started at 08:31:04 AM EDT; Query executed in 0.02 m

,status
0,Table TOT successfully created.


## Tables export

In [29]:
filenames = ['C{latest_year} - NY AND CW 10 YEAR EARNED EXPOSURES BY TIER.csv'.format(latest_year=latest_year),'C{latest_year} - NY AND CW 10 YEAR STATEWIDE EARNED EXPOSURES.csv'.format(latest_year=latest_year)]
create_download_link(['TOTSW','TOT' ], 'SF',  filenames)

Query started at 08:32:52 AM EDT; Query executed in 0.00 m
The following table(s) were created and ready to be downloaded: 
C2019 - NY AND CW 10 YEAR EARNED EXPOSURES BY TIER.csv with 165 rows, 5 columns
C2019 - NY AND CW 10 YEAR STATEWIDE EARNED EXPOSURES.csv with 41 rows, 4 columns



# Recon

For the correct Recon the following files and datasets should be used:
1. NYEXP10_recon.txt
2. NYEXP10_recon2.txt
3. Dataset: 'IPO_DEAL_DB.SB_MORGAN_PERS_LIAB.PL_PH2MRGA1_2019_MF_OUTPUT'

### NYEXP10_recon.txt
201 11737 5866636
202 134 255197
203 498 164416
204 232 125716
211 0 49991
212 0 3210
213 0 5234
214 0 2241
221 65 280404
222 0 11271
223 0 8793
224 4 4117
231 0 1460
232 0 114
233 0 43
234 0 38
271 160 792702
272 0 36332
273 10839 14322663
274 270 1094780
275 43 358735
276 61 213145
280 0 389
281 0 43
763 30 38595
771 0 115048
780 42 141200
781 14 84452
782 35 88296
785 47 206950
786 58 238083
787 72 417221
788 36 222080
901 0 970
911 0 586
921 0 1654
931 0 414
941 0 453
951 0 1314
991 0 0
992 0 84

### NYEXP10_recon2.txt
201 162348 23466544
202 119574 1020788
203 65078 657664
204 18474 502864
211 5 199964
212 0 12840
213 0 20936
214 0 8964
221 2766 1121616
222 1065 45084
223 334 35172
224 225 16468
231 0 5840
232 83 456
233 34 172
234 0 152
271 2637 3170808
272 323 145328
273 214082 57290652
274 25071 4379120
275 28938 1434940
276 2807 852580
280 0 1556
281 0 172
763 1964 154380
771 228 460192
780 4543 564800
781 4917 337808
782 2560 353184
785 4563 827800
786 7357 952332
787 12173 1668884
788 6570 888320
901 10 3880
911 0 2344
921 8 6616
931 0 1656
941 0 1812
951 34 5256
991 0 0
992 2 336

## Recon TOTSW

In [30]:
"""reading input file into pandas df"""
recon_df = pd.read_csv(r"NYEXP10_recon.txt",sep=" ", names=['class_cd','ST_EARN_HOUSE_YR', 'CW_EARN_HOUSE_YR'])

create_table = """ CREATE OR REPLACE TEMP TABLE NYEXP10_recon (
    class_cd NUMBER(5),
    ST_EARN_HOUSE_YR NUMBER (15),
    CW_EARN_HOUSE_YR NUMBER (15)            
    )
    """
%read_sql {create_table}
    
insert_data = """ INSERT INTO NYEXP10_recon VALUES \n"""
for i in range(len(recon_df)-1):
    insert_data += "({class_cd},{stehy}, {cwehy}),\n".format(class_cd = recon_df['class_cd'][i],stehy = recon_df['ST_EARN_HOUSE_YR'][i], cwehy = recon_df['CW_EARN_HOUSE_YR'][i])
insert_data += "({class_cd},{stehy}, {cwehy})".format(class_cd = recon_df['class_cd'][len(recon_df)-1],stehy = recon_df['ST_EARN_HOUSE_YR'][len(recon_df)-1], cwehy = recon_df['CW_EARN_HOUSE_YR'][len(recon_df)-1])
%read_sql {insert_data}

Query started at 08:33:07 AM EDT; Query executed in 0.01 m

,number of rows inserted
0,41


In [32]:
%%read_sql
select a.class_cd, 
a.ST_EARN_HOUSE_YR-b.ST_EARN_HOUSE_YR as ST_EARN_HOUSE_YR_diff,
a.CW_EARN_HOUSE_YR-b.CW_EARN_HOUSE_YR as CW_EARN_HOUSE_YR_diff
from 
(select * from totsw where terr_grp=1) a left join NYEXP10_recon b on a.class_cd=b.class_cd
where a.ST_EARN_HOUSE_YR-b.ST_EARN_HOUSE_YR <>0 or a.CW_EARN_HOUSE_YR-b.CW_EARN_HOUSE_YR<>0

Query started at 08:33:20 AM EDT; Query executed in 0.01 m

,CLASS_CD,ST_EARN_HOUSE_YR_DIFF,CW_EARN_HOUSE_YR_DIFF


In [37]:
qry="""
create or replace temp table totsw_recon_csv as
select a.class_cd, 
a.ST_EARN_HOUSE_YR,b.ST_EARN_HOUSE_YR as ST_EARN_HOUSE_YR_recon,
a.CW_EARN_HOUSE_YR,b.CW_EARN_HOUSE_YR as CW_EARN_HOUSE_YR_recon
from 
(select * from totsw where terr_grp=1) a left join NYEXP10_recon b on a.class_cd=b.class_cd
;"""
%read_sql {qry}
create_download_link(['totsw_recon_csv'],'SF',[f'NYEXP10_totsw_recon_{user_id}_{date}.csv'])

Query started at 08:35:23 AM EDT; Query executed in 0.01 m
The following table(s) were created and ready to be downloaded: 
NYEXP10_totsw_recon_i36484_2022_08_30.csv with 41 rows, 5 columns



## Recon TOT

In [38]:
"""reading input file into pandas df"""
recon_df = pd.read_csv(r"NYEXP10_recon2.txt",sep=" ", names=['class_cd','ST_EARN_HOUSE_YR', 'CW_EARN_HOUSE_YR'])

create_table = """ CREATE OR REPLACE TEMP TABLE NYEXP10_recon2 (
    class_cd NUMBER(5),
    ST_EARN_HOUSE_YR NUMBER (15),
    CW_EARN_HOUSE_YR NUMBER (15)            
    )
    """
%read_sql {create_table}
    
insert_data = """ INSERT INTO NYEXP10_recon2 VALUES \n"""
for i in range(len(recon_df)-1):
    insert_data += "({class_cd},{stehy}, {cwehy}),\n".format(class_cd = recon_df['class_cd'][i],stehy = recon_df['ST_EARN_HOUSE_YR'][i], cwehy = recon_df['CW_EARN_HOUSE_YR'][i])
insert_data += "({class_cd},{stehy}, {cwehy})".format(class_cd = recon_df['class_cd'][len(recon_df)-1],stehy = recon_df['ST_EARN_HOUSE_YR'][len(recon_df)-1], cwehy = recon_df['CW_EARN_HOUSE_YR'][len(recon_df)-1])
%read_sql {insert_data}

Query started at 08:35:57 AM EDT; Query executed in 0.01 m

,number of rows inserted
0,41


In [39]:
%%read_sql
select a.class_cd, 
a.ST_EARN_HOUSE_YR-b.ST_EARN_HOUSE_YR as ST_EARN_HOUSE_YR,
a.CW_EARN_HOUSE_YR-b.CW_EARN_HOUSE_YR as CW_EARN_HOUSE_YR
from 
tot a left join NYEXP10_recon2 b on a.class_cd=b.class_cd
where a.ST_EARN_HOUSE_YR-b.ST_EARN_HOUSE_YR <>0 or a.CW_EARN_HOUSE_YR-b.CW_EARN_HOUSE_YR<>0

Query started at 08:35:57 AM EDT; Query executed in 0.01 m

,CLASS_CD,ST_EARN_HOUSE_YR,CW_EARN_HOUSE_YR


In [41]:
qry="""
create or replace temp table tot_recon_csv as
select a.class_cd, 
a.ST_EARN_HOUSE_YR,b.ST_EARN_HOUSE_YR as ST_EARN_HOUSE_YR_recon,
a.CW_EARN_HOUSE_YR,b.CW_EARN_HOUSE_YR as CW_EARN_HOUSE_YR_recon
from 
tot a left join NYEXP10_recon2 b on a.class_cd=b.class_cd
;"""
%read_sql {qry}
create_download_link(['tot_recon_csv'],'SF',[f'NYEXP10_tot_recon_{user_id}_{date}.csv'])

Query started at 08:37:11 AM EDT; Query executed in 0.00 m
The following table(s) were created and ready to be downloaded: 
NYEXP10_tot_recon_i36484_2022_08_30.csv with 41 rows, 5 columns



# Close Connection

In [ ]:
sf.close_connect()